In [2]:
import numpy as np
### Library needed for flexible data wrangling
import pandas as pd
### Library needed for API requests
import requests
### Ignore needless warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
### FEMA Load Data and inspect
dataset = pd.read_csv("/Users/diego/Independet Study Fall 2021/FemaWebDeclarationAreas.csv")
dataset.head()

,disasterNumber,programTypeCode,programTypeDescription,stateCode,stateName,placeCode,placeName,designatedDate,entryDate,updateDate,closeoutDate,hash,lastRefresh,id
0,1694,DUA,DUA - Disaster Unemployment Assist.,NJ,New Jersey,99031,Passaic (County),2007-04-26T00:00:00.000Z,2007-04-26T00:00:00.000Z,2007-06-11T00:00:00.000Z,2017-08-02T00:00:00.000Z,4d518f033b1ed271cfb7120f1be60ab347bfc66d,2021-07-13T22:47:34.167Z,60c3b899a0ee349d710c8c4f
1,1694,HM,HM - Hazard Mitigation,NJ,New Jersey,99041,Warren (County),2007-05-16T00:00:00.000Z,2007-05-16T00:00:00.000Z,2007-05-16T00:00:00.000Z,2017-08-02T00:00:00.000Z,2f612872234547b80f89d0ee3dd3ac4cd4be99b0,2021-07-13T22:47:34.211Z,60c3b89aa0ee349d710c8ce6
2,1694,OTH,OTH - Other,NJ,New Jersey,99003,Bergen (County),2007-04-26T00:00:00.000Z,2007-04-26T00:00:00.000Z,2007-06-11T00:00:00.000Z,2017-08-02T00:00:00.000Z,e6d90702dc50c85820146072b72411f1a45fe9f4,2021-07-13T22:47:34.228Z,60c3b89aa0ee349d710c8d0c
3,1694,OTH,OTH - Other,NJ,New Jersey,99023,Middlesex (County),2007-04-27T00:00:00.000Z,2007-04-27T00:00:00.000Z,2007-06-11T00:00:00.000Z,2017-08-02T00:00:00.000Z,b6dc2eaeca102bc217b69600ac4e6f5700b695b9,2021-07-13T22:47:34.242Z,60c3b89aa0ee349d710c8d33
4,1694,PA,PA - Public Assistance,NJ,New Jersey,99007,Camden (County),2007-04-27T00:00:00.000Z,2007-04-30T00:00:00.000Z,2007-04-30T00:00:00.000Z,2017-08-02T00:00:00.000Z,4c7cc81e54e60b81c04484477153c545f6c638e1,2021-07-13T22:47:34.255Z,60c3b89aa0ee349d710c8d45


In [4]:
# Wrangling Data
dataset = dataset[["programTypeDescription","stateCode","placeCode","placeName"]]
dataset

,programTypeDescription,stateCode,placeCode,placeName
0,DUA - Disaster Unemployment Assist.,NJ,99031,Passaic (County)
1,HM - Hazard Mitigation,NJ,99041,Warren (County)
2,OTH - Other,NJ,99003,Bergen (County)
3,OTH - Other,NJ,99023,Middlesex (County)
4,PA - Public Assistance,NJ,99007,Camden (County)
...,...,...,...,...
438088,PA-E - Public Buildings,MT,99087,Rosebud (County)
438089,PA-F - Public Utilities,MT,54550,Northern Cheyenne Indian Reservation
438090,PA-F - Public Utilities,MT,99087,Rosebud (County)
438091,PA-G - Recreational or Other,MT,54550,Northern Cheyenne Indian Reservation


In [5]:
dataset = dataset.loc[dataset['programTypeDescription'].
            isin(['DUA - Disaster Unemployment Assist.',
                  'HM - Hazard Mitigation',
                  'PA - Public Assistance'])
            ]
dataset = dataset.loc[dataset['stateCode'].
            isin(['TX',
                  'CO',
                  'NY'])
            ]

In [6]:
dataset = dataset.loc[dataset['placeName'].isin(['New York (County)','Harris (County)','Denver (County)'])]

In [7]:
# create a list of our conditions
conditions = [
    (dataset['programTypeDescription'].isin(['DUA - Disaster Unemployment Assist.'])),
    (dataset['programTypeDescription'].isin(['HM - Hazard Mitigation'])),
    (dataset['programTypeDescription'].isin(['PA - Public Assistance'])),
    ]

# create a list of the values we want to assign for each condition
values = [3,2,1]

# create a new column and use np.select to assign values to it using our lists as arguments
dataset['tier'] = np.select(conditions, values)

# display updated DataFrame
dataset


,programTypeDescription,stateCode,placeCode,placeName,tier
5984,DUA - Disaster Unemployment Assist.,TX,99201,Harris (County),3
6003,HM - Hazard Mitigation,TX,99201,Harris (County),2
18926,DUA - Disaster Unemployment Assist.,TX,99201,Harris (County),3
19010,HM - Hazard Mitigation,TX,99201,Harris (County),2
19125,PA - Public Assistance,TX,99201,Harris (County),1
...,...,...,...,...,...
431301,HM - Hazard Mitigation,NY,99061,New York (County),2
432100,HM - Hazard Mitigation,TX,99201,Harris (County),2
434289,HM - Hazard Mitigation,CO,99031,Denver (County),2
436429,HM - Hazard Mitigation,NY,99061,New York (County),2


In [8]:
### Assign Overall Environmental Risk Score
risk_score = dataset.groupby('placeName')['tier'].sum()
risk_score
risk_score.dtypes
### Notice output is of type integer, need to convert back to dataframe

dtype('int64')

In [9]:
### Convert back to dataframe 
risk_dataframe = pd.DataFrame(risk_score)
### Rename Columns
risk_dataframe = risk_dataframe.rename(columns={"tier": "Risk Score","placeName": "County"})
risk_dataframe

,Risk Score
placeName,
Denver (County),21
Harris (County),121
New York (County),37


In [25]:
### CDC API - Covid Rates
r = requests.get('https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh/data')
print(len(r.text))

787760


In [34]:
dataset2 = pd.read_csv("/Users/diego/Independet Study Fall 2021/all-states-history.csv")
dataset2.head()


,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,NaN,0,NaN,1293.0,1293.0,33.0,0,...,1731628.0,0,NaN,NaN,NaN,NaN,NaN,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,NaN,NaN,119757.0,NaN,2323788.0,2347,NaN,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,NaN,NaN,NaN,481311.0,NaN,0,2736442.0,3380
3,2021-03-07,AS,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,NaN,444089.0,NaN,3842945.0,14856,7908105.0,45110


In [43]:
# Wrangling Data
dataset2 = dataset2[["state","hospitalized"]]
dataset2 = dataset2.dropna()

In [78]:
# Hospitalizations by State Cumulative Throughout the COVID-19 pandemic
hospitalizations = dataset2.groupby('state')['hospitalized'].sum()
### Convert back to dataframe 
hospitalizations = pd.DataFrame(hospitalizations)
hospitalizations.head()

,hospitalized
state,
AK,157626.0
AL,6133645.0
AR,2017857.0
AZ,7425807.0
CO,3478740.0


In [56]:
### Assign three levels of values
### Use numpy to determine distribution of scores
median_hosp = hospitalizations.median()
q3_hosp = np.percentile(hospitalizations, 75)
q1_hosp = np.percentile(hospitalizations, 25)
print(median_hosp,q3_hosp,q1_hosp)

hospitalized    1872346.5
dtype: float64 3666005.25 505407.25


In [76]:
### Assign Scores 
# 50 low to medium hospitalizations < 75%
# 100 high hospitalizations > 75%

# create a list of our conditions
conditions_h = [
    (hospitalizations['hospitalized'] < 3666005.25),
    (hospitalizations['hospitalized'] > 3666005.25)
               ]

# create a list of the values we want to assign for each condition
values_h = [50,100]

# create a new column and use np.select to assign values to it using our lists as arguments
hospitalizations['hospitalization_score'] = np.select(conditions_h, values_h)

# display updated DataFrame
hospitalizations


,hospitalized,hospitalization_score
state,,
AK,157626.0,50
AL,6133645.0,100
AR,2017857.0,50
AZ,7425807.0,100
CO,3478740.0,50
CT,1908439.0,50
FL,13445188.0,100
GA,8971046.0,100
HI,289663.0,50


In [106]:
### Ranking Equation score (1 through 5 based on FEMA + COVID factors)
# -> 1 extremely unsafe; 5 extremely safe 
#Equation: Score = FEMA score + death rate + covid positive rate

dictionary = {'State': ["CO"], 'Climate score': [21], 'COVID score': [50]}
fin_score = pd.DataFrame(data=dictionary)
fin_score

,State,Climate score,COVID score
0,CO,21,50


In [107]:
### Assign final score column
fin_score['Final Score'] = fin_score['Climate score'] + fin_score['COVID score']

In [108]:
### Final Score should be interpreted relative to other states and counties
fin_score

,State,Climate score,COVID score,Final Score
0,CO,21,50,71
